In [1]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
adult = fetch_ucirepo(id=2)

# data (as pandas dataframes)
X = adult.data.features
y = adult.data.targets

In [2]:
import pandas as pd
pd.DataFrame(X)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States
48838,64,NaN,321403,HS-grad,9,Widowed,NaN,Other-relative,Black,Male,0,0,40,United-States
48839,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States
48840,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States


In [3]:
categorical_columns=X.select_dtypes(include='object').columns
categorical_columns

Index(['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'native-country'],
      dtype='object')

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Input, Flatten,Concatenate, Dense
from tensorflow.keras.models import Model
## LabelEncoder
from sklearn.preprocessing import LabelEncoder

inputs_var=[]
num_cat=[]
Embbedings_var=[]
Embbedings_var_flat=[]

le=LabelEncoder()
for col_cat in categorical_columns:
    X[col_cat]=le.fit_transform(X[col_cat])


for col_cat in categorical_columns:
    input_var_temp=Input(shape=(1,))
    inputs_var.append(input_var_temp)
    cant_cat=len(X[col_cat].unique())
    num_cat.append(cant_cat)
    embedding_var_temp = Embedding(input_dim=cant_cat, output_dim=4)(input_var_temp)
    Embbedings_var.append(embedding_var_temp)
    embedding_var_flat_temp = Flatten()(embedding_var_temp)
    Embbedings_var_flat.append(embedding_var_flat_temp)

concatenated_embeddings = Concatenate()(Embbedings_var_flat)

# Dimensiones de la representación codificada
encoding_dim = 8  # Ajusta según necesites

# Codificador
encoded = Dense(64, activation='relu')(concatenated_embeddings)
encoded = Dense(32, activation='relu')(encoded)
encoded = Dense(encoding_dim, activation='relu')(encoded)

# Decodificador
decoded = Dense(32, activation='relu')(encoded)
decoded = Dense(64, activation='relu')(decoded)
decoded_output = Dense(sum(num_cat), activation='sigmoid')(decoded)

# Autoencoder completo
autoencoder = Model(inputs=inputs_var, outputs=decoded_output)

# Compilar el autoencoder
autoencoder.compile(optimizer='adam', loss='mse')  # Puedes usar 'binary_crossentropy' si es más adecuado para tu caso

# Resumen del modelo
autoencoder.summary()


<ipython-input-4-a818f46d04a8>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col_cat]=le.fit_transform(X[col_cat])
<ipython-input-4-a818f46d04a8>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col_cat]=le.fit_transform(X[col_cat])
<ipython-input-4-a818f46d04a8>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 input_3 (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 input_4 (InputLayer)        [(None, 1)]                  0         []                            
                                                                                              

In [ ]:
X_trans = [X[col].values for col in categorical_columns]

autoencoder.fit(X_trans, X_trans, epochs=100, batch_size=32, shuffle=True)

Epoch 1/100
1527/1527 [==============================] - 9s 4ms/step - loss: 11.6973
Epoch 2/100
1527/1527 [==============================] - 6s 4ms/step - loss: 11.5879
Epoch 3/100
1527/1527 [==============================] - 5s 3ms/step - loss: 11.5879
Epoch 4/100
1527/1527 [==============================] - 6s 4ms/step - loss: 11.5879
Epoch 5/100
1527/1527 [==============================] - 5s 3ms/step - loss: 11.5879
Epoch 6/100
1527/1527 [==============================] - 6s 4ms/step - loss: 11.5879
Epoch 7/100
1527/1527 [==============================] - 5s 3ms/step - loss: 11.5879
Epoch 8/100
1527/1527 [==============================] - 6s 4ms/step - loss: 11.5879
Epoch 9/100
1527/1527 [==============================] - 5s 3ms/step - loss: 11.5879
Epoch 10/100
1527/1527 [==============================] - 6s 4ms/step - loss: 11.5879
Epoch 11/100
1527/1527 [==============================] - 6s 4ms/step - loss: 11.5879
Epoch 12/100
1527/1527 [==============================] - 5s 3m